In [1]:
# import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import time
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt

# API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2017)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E", "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", 
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.head(100)

census_pd["Zipcode"] = census_pd["Zipcode"].astype('int64')

phoenix = census_pd.loc[census_pd["Zipcode"] == "85253"]
phoenix
census_pd.dtypes

33120


C:\Users\LCDR-Data\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Zipcode                int64
Population           float64
Median Age           float64
Household Income     float64
Per Capita Income    float64
Poverty Count        float64
Poverty Rate         float64
dtype: object

In [3]:
file_to_load = "covid_zip"

# Read Purchasing File and store into Pandas data frame
covid_df = pd.read_csv(file_to_load)
covid_df = covid_df.rename(columns = {"POSTCODE" : "Zipcode"})
covid_df.dtypes

Unnamed: 0                 int64
Zipcode                    int64
ConfirmedCaseCategory     object
ConfirmedCaseCount       float64
dtype: object

In [4]:
merged_df = pd.merge(covid_df, census_pd, on="Zipcode", how="left")
merged_df.drop(columns=["Unnamed: 0"])

,Zipcode,ConfirmedCaseCategory,ConfirmedCaseCount,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,85003,Greater Than 250,277.0,8694.0,39.2,47554.0,36513.0,1662.0,19.116632
1,85004,101-250,206.0,5944.0,32.3,33862.0,31679.0,1772.0,29.811575
2,85006,Greater Than 250,837.0,27671.0,31.0,32166.0,16583.0,10919.0,39.460085
3,85007,Greater Than 250,639.0,14810.0,33.2,31396.0,20438.0,5273.0,35.604321
4,85008,Greater Than 250,2415.0,60700.0,28.7,37696.0,17694.0,18762.0,30.909390
...,...,...,...,...,...,...,...,...,...
405,86514,Data Suppressed,NaN,2734.0,37.6,24199.0,12098.0,1209.0,44.220922
406,86515,0,0.0,4933.0,33.8,35417.0,13441.0,1473.0,29.860126
407,86515,Data Suppressed,NaN,4933.0,33.8,35417.0,13441.0,1473.0,29.860126
408,86538,Data Suppressed,NaN,2080.0,28.9,26304.0,11644.0,888.0,42.692308
